In [31]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from datetime import datetime

ticker1 = 'SSO'          # 'UPRO'
ticker2 = 'SDS'           # 'SPXU'
underlying_asset = '^GSPC'  # '^GSPC'
start_date = '2009-06-23'
end_date = '2025-12-31'

tickers = [ticker1, ticker2, underlying_asset]
data = yf.download(tickers, start=start_date, end=end_date, progress=False)

# For reference
#print("Columns in data:", data.columns)
#print("First few rows of data:\n", data.head())

if isinstance(data.columns, pd.MultiIndex):
    adj_close = pd.DataFrame({
        'ticker1': data[('Close', ticker1)],
        'ticker2': data[('Close', ticker2)],
        'underlying_asset': data[('Close', underlying_asset)]
    })
else:
    adj_close = data[[ticker1, ticker2, underlying_asset]]
returns = adj_close.pct_change().dropna()
returns.columns = ['ticker1', 'ticker2', 'underlying_asset']

# Prepare data for regression
X = returns['underlying_asset']  # Independent variable (underlying asset returns)
X = sm.add_constant(X)           # Add intercept term

# Dependent variables (ticker1 and ticker2 returns)
y_ticker1 = returns['ticker1']
y_ticker2 = returns['ticker2']

# Perform OLS regression for ticker1
model_ticker1 = sm.OLS(y_ticker1, X).fit()
alpha_ticker1 = model_ticker1.params['const']
beta_ticker1 = model_ticker1.params['underlying_asset']

# Perform OLS regression for ticker2
model_ticker2 = sm.OLS(y_ticker2, X).fit()
alpha_ticker2 = model_ticker2.params['const']
beta_ticker2 = model_ticker2.params['underlying_asset']

# Print results
print(f"\n{ticker1} Regression Results:")
print(f"Alpha (Intercept): {alpha_ticker1:.6f}")
print(f"Beta (Empirical Leverage): {beta_ticker1:.3f}")
print(f"R-squared: {model_ticker1.rsquared:.3f}")
print(f"P-value for Beta: {model_ticker1.pvalues['underlying_asset']:.4f}\n")

print(f"{ticker2} Regression Results:")
print(f"Alpha (Intercept): {alpha_ticker2:.6f}")
print(f"Beta (Empirical Leverage): {beta_ticker2:.3f}")
print(f"R-squared: {model_ticker2.rsquared:.3f}")
print(f"P-value for Beta: {model_ticker2.pvalues['underlying_asset']:.4f}")

# Compare to target leverage (adjust these based on your LETFs, e.g., 3 for ticker1, -3 for ticker2)
target_leverage_ticker1 = 2 # When running this for general code, don't hardcode it but automatically assign the target leverage from yfinance
target_leverage_ticker2 = -2 
print("\nComparison to Target Leverage:")
print(f"{ticker1} Target Leverage: {target_leverage_ticker1}, Empirical Leverage: {beta_ticker1:.3f}, Difference: {beta_ticker1 - target_leverage_ticker1:.3f}")
print(f"{ticker2} Target Leverage: {target_leverage_ticker2}, Empirical Leverage: {beta_ticker2:.3f}, Difference: {beta_ticker2 - target_leverage_ticker2:.3f}")


SSO Regression Results:
Alpha (Intercept): 0.000049
Beta (Empirical Leverage): 1.983
R-squared: 0.997
P-value for Beta: 0.0000

SDS Regression Results:
Alpha (Intercept): -0.000050
Beta (Empirical Leverage): -1.982
R-squared: 0.997
P-value for Beta: 0.0000

Comparison to Target Leverage:
SSO Target Leverage: 2, Empirical Leverage: 1.983, Difference: -0.017
SDS Target Leverage: -2, Empirical Leverage: -1.982, Difference: 0.018
